In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn import metrics
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from keras.layers import Dense, Embedding, LSTM, BatchNormalization, Bidirectional, LayerNormalization
from sklearn.utils import resample
from matplotlib import pyplot
from numpy import mean
from numpy import std
from numpy import array
from sklearn.metrics import max_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn import ensemble

In [36]:
def evaluate_model(X_train_res, Y_train_res, X_test,Y_test):
    
    model = Sequential()
# LSTM LAYER IS ADDED TO MODEL WITH 128 CELLS IN IT
    model.add(LSTM(128, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))
    model.add(Dropout(0.2))  # 20% DROPOUT ADDED FOR REGULARIZATION
    model.add(BatchNormalization())
    model.add(LSTM(128, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(LSTM(64, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(LSTM(64, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(LSTM(30, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))  # FINAL CLASSIFICATION LAYER WITH 2 CLASSES AND SOFTMAX
    
    # OPTIMIZER SETTINGS
    opt = tf.keras.optimizers.Adam(learning_rate=0.0017, decay=1e-6)

    # MODEL COMPILE
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    model.fit(X_train_res, Y_train_res, epochs=5, batch_size=512, verbose=1,validation_data=(X_test, Y_test))
                                                      #callbacks=[early_stop, tfdocs.modeling.EpochDots()])
    return model

In [37]:
ann_estimator = KerasRegressor(build_fn= evaluate_model, epochs=100, batch_size=10, verbose=1)

C:\Users\ILLEGEAR\AppData\Local\Temp\ipykernel_13716\1340795224.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ann_estimator = KerasRegressor(build_fn= evaluate_model, epochs=100, batch_size=10, verbose=1)


In [18]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.ensemble import AdaBoostRegressor

In [43]:
boosted_ann = AdaBoostRegressor(base_estimator= ann_estimator)
boosted_ann.fit(X_train_res, Y_train_res, X_train_res)# scale your training data 
boosted_ann.predict(X_test)

ValueError: Found array with dim 3. None expected <= 2.

In [3]:
data4 = pd.read_csv('data4.csv')

In [4]:
data4 = data4.drop(['Unnamed: 0'], axis = 1)

In [5]:
train,test= train_test_split(data4, test_size=0.3,random_state=123, stratify=data4.Result)# stratify the outcome
X_train =train.drop(['Result'], axis = 1)
X_test = test.drop(['Result'], axis = 1)
Y_train = train['Result'].values
Y_test = test['Result'].values

In [6]:
print("Before OverSampling, counts of label '1': {}".format(sum(Y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(Y_train == 0)))
  
# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X_train_res, Y_train_res = sm.fit_resample(X_train, Y_train.ravel())
  
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(Y_train_res.shape))
  
print("After OverSampling, counts of label '1': {}".format(sum(Y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(Y_train_res == 0)))

Before OverSampling, counts of label '1': 202311
Before OverSampling, counts of label '0': 721368 

After OverSampling, the shape of train_X: (1442736, 63)
After OverSampling, the shape of train_y: (1442736,) 

After OverSampling, counts of label '1': 721368
After OverSampling, counts of label '0': 721368


In [7]:
X_train_res= np.expand_dims(X_train_res, axis=2)

In [8]:
X_train_res.shape

(1442736, 63, 1)

In [9]:
X_test = np.expand_dims(X_test, axis=2)

In [10]:
X_test.shape

(395863, 63, 1)